In [1]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import heapq as hp
import time

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/dnrocha1/information_retrieval/master/lab02/data/results.csv')

# Pré-processamento dos dados

Os dados serão tratados da mesma forma que os exercícios anteriores: os conteúdos dos documentos tiveram as letras transformadas para minúsculas, contendo caractéres alfanuméricos. Novamente, os tokens criados consideram apenas palavras com tamanho maior ou igual a três e inclui termos que possuam hífen e apóstrofo.

In [3]:
documents = data['text'].apply(lambda x: x.lower())

regex = RegexpTokenizer(r'\b[A-zÀ-ú-\'\d]{3,}')
# tokens = regex.tokenize(texts)

# Questão 1

### Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.

O índice gerado utiliza um dicionário para guardar as informações necessárias. A chave será composta pelos tokens produzidos, enquanto o respectivo valor será uma lista de tuplas. Cada tupla é definida pelo id do documento e pela frequência da palavra dentro desse documento.

In [4]:
doc_list = []

def build_index(documents=documents):
  inverted_list = {}

  n_doc = 0
  for document in documents:
    n_doc += 1
    doc_list.append(n_doc)
    token = regex.tokenize(document)
    counter = list(Counter(token).items())
    for elem in counter:
      key = elem[0]
      freq = elem[1]
      if key in inverted_list.keys():
        if n_doc not in inverted_list[key][0]:
          inverted_list[key].append((n_doc,freq))
      else:
        inverted_list[key] = [(n_doc,freq)]
  
  return inverted_list

index = build_index()

O trecho abaixo corresponde ao código necessário para guardar o índice construído no disco, no formato `.csv`.

In [5]:
# Guarde o índice em disco em formato csv

csv = pd.DataFrame()
csv['tokens'] = list(index.keys())
csv['postings'] = list(index.values())

csv.to_csv('index.csv', index=False)

# Questão 2
### Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18).

Abaixo estão as implementações das duas abordagens vistas no livro. A função `get_doc_list` é utilizada dentro da implementação do processamento `document_at_time`. Isso é necessário para recuperar todos os documentos existentes em `index`, que é passado como parâmetro. 

In [6]:
%%capture

def get_doc_list(postings):
  
  def filter_n_doc(postings=postings):
    return [[e[0] for e in elem] for elem in postings]
  
  docs = filter_n_doc()
  flat_list = [item for sublist in docs for item in sublist]
  return list(set(flat_list))
    

get_doc_list(index.values())

Implementação do processamento documento-por-vez. A variável `priority_queue` utiliza uma lista que é transformada em fila de prioridade, de forma similar ao pseudocódigo desse algoritmo.

In [7]:
def document_at_time(query, index, k):
  query_words = query.split()    
  inverted_lists = []
  priority_queue = []

  for word in query_words:
    if word in index.keys():
      inverted_lists.append(index[word])

  doc_list = get_doc_list(index.values())
  for d in doc_list:
    score = 0
    for inv in inverted_lists:
      for posting in inv:
        if d == posting[0]:
          score += posting[1]
    if score != 0:
      priority_queue.append((score,d))
    
  hp._heapify_max(priority_queue)
  
  top_k = []
  
  for i in range(1,k+1):
    if priority_queue != []:
      top = hp._heappop_max(priority_queue)
      top_k.append(top)
  
  return top_k

# document_at_time("juíza federal", index, 10)

Implementação do processamento termo-por-vez. A fila de prioridade é utilizada de forma similar ao algoritmo anterior. Aqui é definido um dicionário `acc`, que serve para registrar os acumuladores dos documentos. Nesse caso, a chave será o id do documento, enquanto o valor será o score deste documento.

In [8]:
def term_at_time(query, index, k):
  query_words = query.split()    
  inverted_lists = []
  acc = {}

  for word in query_words:
    if word in index.keys():
      inverted_lists.append(index[word])

  for lst in inverted_lists:
    for posting in lst:
      d = posting[0]
      freq = posting[1]
      if d in acc.keys():
        acc[d] = acc[d] + freq
      else:
        acc[d] = freq
  
  priority_queue = list(map(lambda elem: (elem[1],elem[0]), acc.items()))
  
  hp._heapify_max(priority_queue)
  
  top_k = []
  
  for i in range(1,k+1):
    if priority_queue != []:
      top = hp._heappop_max(priority_queue)
      top_k.append(top)
  
  return top_k

# term_at_time("juíza federal", index, 10)

### Defina 5 consultas de um termo somente.

As cinco consultas definidas são atribuídas à variável `queries`.

In [9]:
queries = ["educação","polícia","cortes","presidente","segurança"]

### Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).

A execuçao de cada uma das implementações foi feita e o k definido como sendo igual a 10. Os resultados das consultas foram guardados em listas, para depois serem comparados. De modo similar, os tempos de processamento das consultas também foram salvos.

### Dê evidências de que sua implementação está correta e compare os tempos médios de execução e uso de memória de cada algoritmo.

In [10]:
results_document = []
results_term = []
k = 10

doc_times = []
term_times = []

for query in queries:
  start = time.time()
  d = document_at_time(query, index, k)
  end = time.time()
  
  results_document.append(d)  
  doc_times.append(end - start)
  
  start = time.time()
  t = term_at_time(query, index, k)
  end = time.time()
  
  results_term.append(t)
  term_times.append(end - start)

### Dê evidências de que sua implementação está correta e compare os tempos médios de execução e uso de memória de cada algoritmo.

A corretude das implementações corresponde a comparação dos resultados de cada uma. Isso corresponde à coluna `compare` da tabela abaixo. Como podemos observar, todos os resultados dessa variável são equivalentes. 

In [11]:
pd.options.display.max_colwidth = 120

df = pd.DataFrame()

df['query'] = queries
df['document_at_time'] = results_document
df['term_at_time'] = results_term
df['compare'] = df.document_at_time == df.term_at_time
df['time_DAAT'] = doc_times
df['time_TAAT'] = term_times

df

,query,document_at_time,term_at_time,compare,time_DAAT,time_TAAT
0,educação,"[(22, 221), (11, 222), (7, 130), (6, 239), (5, 160), (5, 37), (4, 215), (4, 110), (3, 233), (3, 205)]","[(22, 221), (11, 222), (7, 130), (6, 239), (5, 160), (5, 37), (4, 215), (4, 110), (3, 233), (3, 205)]",True,0.018773,0.000026
1,polícia,"[(8, 151), (4, 214), (4, 93), (3, 241), (3, 150), (3, 65), (2, 249), (2, 230), (2, 207), (2, 181)]","[(8, 151), (4, 214), (4, 93), (3, 241), (3, 150), (3, 65), (2, 249), (2, 230), (2, 207), (2, 181)]",True,0.052742,0.000021
2,cortes,"[(2, 136), (1, 217), (1, 203), (1, 138), (1, 98), (1, 94), (1, 37), (1, 20)]","[(2, 136), (1, 217), (1, 203), (1, 138), (1, 98), (1, 94), (1, 37), (1, 20)]",True,0.017621,0.000015
3,presidente,"[(16, 166), (15, 63), (12, 151), (11, 216), (11, 19), (9, 205), (9, 86), (8, 25), (7, 174), (6, 235)]","[(16, 166), (15, 63), (12, 151), (11, 216), (11, 19), (9, 205), (9, 86), (8, 25), (7, 174), (6, 235)]",True,0.019398,0.000047
4,segurança,"[(6, 239), (6, 65), (3, 151), (3, 134), (3, 12), (2, 247), (2, 179), (2, 153), (2, 150), (2, 143)]","[(6, 239), (6, 65), (3, 151), (3, 134), (3, 12), (2, 247), (2, 179), (2, 153), (2, 150), (2, 143)]",True,0.017339,0.000043


In [12]:
print("O tempo médio do documento-por-vez foi {:.10f}, enquanto o termo-por-vez foi de {:.10f}".format(sum(doc_times)/len(queries), sum(term_times)/len(queries)))

O tempo médio do documento-por-vez foi 0.0251746655, enquanto o termo-por-vez foi de 0.0000304699


Em relação aos tempos de execução, verifica-se que o tempo médio do documento-por-vez é muito maior do que do termo-por-vez. Isso pode acontecer por conta dele precisar recuperar todos documentos do índice, o que acaba sendo muito custoso.

# Questão 3
### Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).

A implementação feita foi a do documento-por-vez em sua versão conjuntiva. Nele, foram combinados todos os termos das listas invertidas que recuperados anteriormente. Feito isso, o objetivo é garantir que o documento `d` aparece em todos as listas invertidas, para que todos os termos da consulta estejam presentes em um determinado documento.

In [13]:
def document_at_time_conj(query, index, k):

  query_words = query.split()    
  inverted_lists = []
  priority_queue = []

  for word in query_words:
    if word in index.keys():
      inverted_lists.append(index[word])

  merged = [item for sublist in inverted_lists for item in sublist]
  merged.sort()

  for i in range(len(merged)):
    score = 0
    d = merged.pop()
    count = 1
    for posting in merged:
      if posting[0] == d[0]:
        score = score + posting[1]
        count += 1
      if score != 0 and count == len(inverted_lists):
        score += d[1]
        priority_queue.append((score,d[0]))

  hp._heapify_max(priority_queue)

  top_k = []

  for i in range(1,k+1):
    if priority_queue != []:
      top = hp._heappop_max(priority_queue)
      top_k.append(top)

  return top_k

# document_at_time_conj("segurança federal",index,10)

### Defina 5 consultas conjuntivas (AND).

In [14]:
queries_conj = ["governo federal", "jair bolsonaro", "forças armadas", "ditadura militar", "golpe estado", "juíza federal"]

### Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).

In [15]:
results = []
k = 10

for query in queries_conj:
  d = document_at_time_conj(query, index, k)
  results.append(d)

In [16]:
pd.options.display.max_colwidth = 120

df = pd.DataFrame()

df['query'] = queries_conj
df['results'] = results

df

,query,results
0,governo federal,"[(19, 173), (14, 166), (13, 248), (12, 115), (11, 229), (10, 225), (10, 138), (9, 205), (9, 37), (8, 208)]"
1,jair bolsonaro,"[(52, 151), (48, 207), (39, 166), (12, 216), (6, 238), (6, 237), (5, 25), (4, 222), (3, 231), (3, 220)]"
2,forças armadas,"[(15, 150), (9, 25), (8, 208), (8, 166), (6, 151), (6, 6), (6, 1), (5, 249), (5, 65), (4, 115)]"
3,ditadura militar,"[(25, 7), (18, 115), (16, 238), (15, 223), (15, 216), (13, 25), (12, 208), (12, 165), (12, 95), (11, 172)]"
4,golpe estado,"[(14, 25), (8, 166), (8, 3), (7, 165), (6, 208), (6, 151), (6, 1), (5, 233), (4, 114), (4, 98)]"
5,juíza federal,"[(4, 1)]"


### Dê evidências de que sua implementação está correta.

Isso pode ser feito por meio de uma consulta com mais termos e observando o índice invertido de cada um dos termos da consulta.

In [17]:
query = "aniversário das forças armadas"
terms = query.split()

postings = [index[term] for term in terms]

document_at_time_conj(query, index, 10)

[(14, 6)]

In [18]:
pd.options.display.max_colwidth = 500

df = pd.DataFrame()

df['terms'] = terms
df['inv_list'] = postings

df

,terms,inv_list
0,aniversário,"[(1, 1), (6, 2), (20, 1), (34, 1), (80, 1), (94, 1), (169, 2), (172, 5), (174, 1), (175, 2), (228, 1)]"
1,das,"[(2, 1), (3, 3), (4, 1), (6, 6), (7, 5), (8, 3), (9, 2), (11, 8), (12, 8), (13, 2), (14, 1), (15, 2), (16, 2), (17, 3), (18, 2), (19, 4), (20, 6), (21, 2), (22, 4), (23, 3), (24, 2), (25, 6), (26, 1), (27, 4), (28, 1), (29, 5), (30, 4), (31, 1), (32, 6), (33, 4), (34, 6), (35, 1), (36, 2), (37, 5), (38, 1), (39, 1), (40, 1), (41, 2), (42, 2), (43, 3), (46, 1), (47, 3), (48, 1), (49, 1), (50, 3), (51, 1), (53, 1), (54, 3), (55, 3), (56, 6), (57, 1), (59, 4), (60, 4), (61, 2), (63, 10), (64, 2..."
2,forças,"[(1, 3), (6, 3), (7, 1), (12, 1), (19, 3), (20, 1), (25, 5), (42, 1), (63, 2), (65, 3), (73, 1), (94, 1), (98, 1), (99, 2), (107, 1), (115, 2), (150, 8), (151, 4), (162, 1), (165, 1), (166, 4), (167, 1), (173, 1), (182, 1), (195, 1), (205, 1), (208, 4), (209, 1), (210, 3), (218, 2), (223, 1), (228, 1), (245, 3), (246, 1), (247, 1), (248, 2), (249, 4)]"
3,armadas,"[(1, 3), (6, 3), (12, 1), (25, 4), (42, 1), (65, 2), (99, 2), (115, 2), (150, 7), (151, 2), (156, 1), (165, 1), (166, 4), (205, 1), (208, 4), (218, 1), (223, 1), (246, 1), (249, 1)]"
